In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19, DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
import os
import re

# Updated dataset paths for guava
train_dir = './images/train'
test_dir = './images/test'
validation_dir = './images/val'

# Data augmentation for training data
train_datagen = ImageDataGenerator(rescale=1./255)

# Only rescaling for validation and test data
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Load data from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = val_test_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# Define checkpoint path for guava
checkpoint_dir = "./checkpoints_guava"
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_path = os.path.join(checkpoint_dir, "model_checkpoint_guava_epoch_{epoch:02d}.h5")

# Load base models without the top layers
input_tensor = Input(shape=(224, 224, 3))
vgg19_base = VGG19(weights='imagenet', include_top=False, input_tensor=input_tensor)
densenet_base = DenseNet121(weights='imagenet', include_top=False, input_tensor=input_tensor)

# Freeze all base model layers initially
for layer in vgg19_base.layers:
    layer.trainable = False
for layer in densenet_base.layers:
    layer.trainable = False

# Extract features
vgg19_output = GlobalAveragePooling2D()(vgg19_base.output)
densenet_output = GlobalAveragePooling2D()(densenet_base.output)

# Concatenate features
merged = Concatenate()([vgg19_output, densenet_output])
merged = Dense(1024, activation='relu')(merged)
predictions = Dense(1, activation='sigmoid')(merged)

# Define final model
model = Model(inputs=input_tensor, outputs=predictions)

# Find latest checkpoint for guava
def get_latest_checkpoint():
    checkpoint_files = [f for f in os.listdir(checkpoint_dir) if f.startswith("model_checkpoint_guava_epoch")]
    if not checkpoint_files:
        return None, 0

    # Extract epoch numbers
    epochs_trained = [int(re.search(r'epoch_(\d+)', f).group(1)) for f in checkpoint_files]
    latest_epoch = max(epochs_trained)
    latest_checkpoint = os.path.join(checkpoint_dir, f"model_checkpoint_guava_epoch_{latest_epoch:02d}.h5")
    return latest_checkpoint, latest_epoch

latest_checkpoint, last_trained_epoch = get_latest_checkpoint()

# Load the latest model checkpoint if available
if latest_checkpoint and os.path.exists(latest_checkpoint):
    print(f"Resuming training from checkpoint: {latest_checkpoint}")
    model = load_model(latest_checkpoint)
else:
    print("No saved model found. Starting fresh.")

# Compile model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# ModelCheckpoint callback for guava
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_best_only=False,
    save_weights_only=False,
    verbose=1
)

# Training Phases
epochs_phase_1 = 10
epochs_phase_2 = 5
epochs_phase_3 = 5

# Determine phase and continue training
if last_trained_epoch < epochs_phase_1:
    print(f"Starting Phase 1 (continuing from epoch {last_trained_epoch + 1})")
    history1 = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // validation_generator.batch_size,
        initial_epoch=last_trained_epoch,
        epochs=epochs_phase_1,
        callbacks=[checkpoint_callback]
    )
    last_trained_epoch = epochs_phase_1

# Unfreeze top 5 layers and continue training
if last_trained_epoch < epochs_phase_1 + epochs_phase_2:
    for layer in vgg19_base.layers[-5:]:
        layer.trainable = True
    for layer in densenet_base.layers[-5:]:
        layer.trainable = True

    model.compile(
        optimizer=Adam(learning_rate=1e-5),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    print(f"Starting Phase 2 (continuing from epoch {last_trained_epoch + 1})")
    history2 = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // validation_generator.batch_size,
        initial_epoch=last_trained_epoch,
        epochs=epochs_phase_1 + epochs_phase_2,
        callbacks=[checkpoint_callback]
    )
    last_trained_epoch = epochs_phase_1 + epochs_phase_2

# Unfreeze all layers and fine-tune
if last_trained_epoch < epochs_phase_1 + epochs_phase_2 + epochs_phase_3:
    for layer in vgg19_base.layers:
        layer.trainable = True
    for layer in densenet_base.layers:
        layer.trainable = True

    model.compile(
        optimizer=Adam(learning_rate=1e-6),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    print(f"Starting Phase 3 (continuing from epoch {last_trained_epoch + 1})")
    history3 = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // validation_generator.batch_size,
        initial_epoch=last_trained_epoch,
        epochs=epochs_phase_1 + epochs_phase_2 + epochs_phase_3,
        callbacks=[checkpoint_callback]
    )

# Evaluate model
test_loss, test_accuracy = model.evaluate(test_generator)
val_loss, val_accuracy = model.evaluate(validation_generator)

print(f'Test Loss (Guava): {test_loss}')
print(f'Test Accuracy (Guava): {test_accuracy}')
print(f'Validation Loss (Guava): {val_loss}')
print(f'Validation Accuracy (Guava): {val_accuracy}')

Found 2224 images belonging to 2 classes.
Found 478 images belonging to 2 classes.
Found 479 images belonging to 2 classes.
Resuming training from checkpoint: ./checkpoints_guava\model_checkpoint_guava_epoch_17.h5


Starting Phase 3 (continuing from epoch 18)
Epoch 18/20


C:\Users\kavit\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.9879 - loss: 0.0561
Epoch 18: saving model to ./checkpoints_guava\model_checkpoint_guava_epoch_18.h5


69/69 ━━━━━━━━━━━━━━━━━━━━ 792s 11s/step - accuracy: 0.9879 - loss: 0.0562 - val_accuracy: 0.9754 - val_loss: 0.0872
Epoch 19/20
 1/69 ━━━━━━━━━━━━━━━━━━━━ 11:08 10s/step - accuracy: 0.9688 - loss: 0.0840

C:\Users\kavit\anaconda3\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 19: saving model to ./checkpoints_guava\model_checkpoint_guava_epoch_19.h5


69/69 ━━━━━━━━━━━━━━━━━━━━ 149s 2s/step - accuracy: 0.9688 - loss: 0.0840 - val_accuracy: 0.9799 - val_loss: 0.0816
Epoch 20/20
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.9824 - loss: 0.0684
Epoch 20: saving model to ./checkpoints_guava\model_checkpoint_guava_epoch_20.h5


69/69 ━━━━━━━━━━━━━━━━━━━━ 441s 6s/step - accuracy: 0.9824 - loss: 0.0683 - val_accuracy: 0.9799 - val_loss: 0.0797
15/15 ━━━━━━━━━━━━━━━━━━━━ 73s 5s/step - accuracy: 0.9876 - loss: 0.0699
15/15 ━━━━━━━━━━━━━━━━━━━━ 73s 5s/step - accuracy: 0.9729 - loss: 0.0884
Test Loss (Guava): 0.0858822911977768
Test Accuracy (Guava): 0.9770354628562927
Validation Loss (Guava): 0.08687326312065125
Validation Accuracy (Guava): 0.9748954176902771
